In [22]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
import os

from selenium.webdriver.chrome.options import Options
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
from datetime import datetime
import time
import pandas as pd
import numpy as np
# import cx_Oracle
import re
from tqdm import tqdm
from IPython import display

import urllib

In [23]:
naver_df = pd.read_csv('음식점완성2.csv', encoding='cp949')
naver_df = naver_df[['콘텐츠명']]
naver_df['naver_keyword'] = '제주도%20' + naver_df['콘텐츠명']
naver_df['naver_keyword'] = naver_df['naver_keyword'].str.replace(' ', '%20')
naver_df

,콘텐츠명,naver_keyword
0,달빛머금은카페,제주도%20달빛머금은카페
1,해적선,제주도%20해적선
2,영춘식당,제주도%20영춘식당
3,멘도롱 카페,제주도%20멘도롱%20카페
4,열공부엌,제주도%20열공부엌
...,...,...
1575,바다풍경횟집,제주도%20바다풍경횟집
1576,우리회수산,제주도%20우리회수산
1577,제주선,제주도%20제주선
1578,우무깨식당,제주도%20우무깨식당


In [24]:
naver_df = naver_df[200:300]
naver_df

,콘텐츠명,naver_keyword
200,해녀3대할망네,제주도%20해녀3대할망네
201,서바당횟집,제주도%20서바당횟집
202,초승달지대,제주도%20초승달지대
203,세자매식당,제주도%20세자매식당
204,제리스펍,제주도%20제리스펍
...,...,...
295,루스트플레이스 아라점,제주도%20루스트플레이스%20아라점
296,청정고을식당,제주도%20청정고을식당
297,준클라시코,제주도%20준클라시코
298,메로식당,제주도%20메로식당


In [25]:
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
naver_driver = webdriver.Chrome(service=service, options=options)

In [26]:
all_data = [] # 모든 데이터를 담을 리스트
for i, keyword in enumerate(naver_df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {naver_df.shape[0] -1} 행", keyword)

    nic_list = []
    re_list = []
    date_list = []

    try:
        naver_driver.get(f'https://map.naver.com/v5/search/{keyword}\n')
        naver_driver.implicitly_wait(5)
        
        try:
            print('entryIframe 체크')
            naver_driver.switch_to.frame('entryIframe')
            naver_driver.implicitly_wait(5)
            print('키워드4' + naver_df.iloc[i]['콘텐츠명'])
            
            # 가게이름
            if naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text == naver_df.iloc[i]['콘텐츠명']:
                print('Fc1rA 있음')
                naver_driver.implicitly_wait(3)
                name = naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text
                print(name)

                # 카테고리 수집
                try:
                    naver_driver.implicitly_wait(3)
                    cate = naver_driver.find_element(By.CLASS_NAME,'DJJvD').text
                    naver_driver.implicitly_wait(3)
                    print(cate)

                except:
                    print('카테고리없음')
                    cate = ''

                # 별점 수집
                try:
                    naver_driver.implicitly_wait(3)
                    star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
                    naver_driver.implicitly_wait(3)
                    print(star)

                except:
                    print('별점없음')
                    star = ''

                # 주소 수집
                try:
                    naver_driver.implicitly_wait(3)
                    addr = naver_driver.find_element(By.CLASS_NAME,'LDgIH').text
                    naver_driver.implicitly_wait(3)
                    print(addr)

                except:
                    print('주소없음')
                    addr = ''

                # 운영시간 수집
                try:
                    naver_driver.implicitly_wait(3)
                    naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/a').send_keys('\n')
                    naver_driver.implicitly_wait(3)
                    oper = naver_driver.find_element(By.CLASS_NAME,'i8cJw').text
                    print(oper)
                    oper_time = naver_driver.find_element(By.CLASS_NAME,'H3ua4').text
                    print(oper_time)
                    open_close = oper+''+oper_time

                except:
                    print('운영시간없음')
                    open_close = ''

                # 전화번호 수집
                try:
                    naver_driver.implicitly_wait(3)
                    tel = naver_driver.find_element(By.CLASS_NAME,'xlx7Q').text
                    naver_driver.implicitly_wait(3)
                    print(tel)

                except:
                    print('전화번호없음')
                    tel = ''

                # 리뷰 수집
                try:
                    naver_driver.implicitly_wait(3)
                    naver_driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]").click()
                    print('리뷰 클릭')
                        
                    # 더보기 클릭
                    try:
                        while True:
                            naver_driver.implicitly_wait(3)
                            naver_driver.find_element(By.CLASS_NAME,'fvwqf').click()
                            print('더보기 클릭')

                            try:
                                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                while True:
                                    naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                                    time.sleep(2)

                                    scroll_height = naver_driver.execute_script("return document.body.scrollHeight")

                                    if scroll_location == scroll_height:
                                        break

                                    else:
                                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                    naver_driver.implicitly_wait(3)
                                    print('스크롤 내림')

                            except:
                                print('스크롤 안됨')

                            if not naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]'):
                                break
                                
                    except:
                        print('더보기 안됨')
                        
                    # 리뷰 더보기 클릭
                    try:
                        for button in naver_driver.find_elements(By.CSS_SELECTOR,'div.ZZ4OK > a.xHaT3'):
                            button.send_keys('\n')
                            naver_driver.implicitly_wait(3)
                            print('리뷰 더보기 클릭')

                    except:
                        print('리뷰 더보기 없음')

                    # 이미지, 리뷰 소스 가져오기
                    try:
                        naver_driver.find_element(By.CLASS_NAME,'sBWyy')
                        naver_driver.implicitly_wait(3)
                        print('sBWyy 있음')

                        naver_html = naver_driver.page_source
                        naver_soup = BeautifulSoup(naver_html, 'lxml')
                        naver_driver.implicitly_wait(3)
                        print('페이지 소스 가져옴')
                        
                        # 이미지 수집
                        try:
                            img = naver_soup.find('div', attrs = {'id':'ibu_1'})
                            naver_driver.implicitly_wait(3)
                            img_link = re.findall('["](https.*[.].*g)["]', str(img))
                            print(img_link)

                        except:
                            print('이미지없음')
                            img_link = ''
                            
                        try:
                            for lst in naver_soup.find_all('li', attrs = {'class':'YeINN'}):
                                try:
                                    nic = lst.find('div', attrs = {'class':'sBWyy'}).text # 변수명 할당
                                    print(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                    nic_list.append(nic) # nic_list에 추가
                                except:
                                    print('닉네임 없음')
                                    nic_list.append('')


                                try:
                                    naver_driver.implicitly_wait(3)
                                    re = lst.find('span', attrs = {'class':'zPfVt'}).text # 변수명 할당
                                    print(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                    re_list.append(re)
                                except:
                                    print('글 리뷰없음')
                                    re_list.append('')
                                    
                                try:
                                    naver_driver.implicitly_wait(3)   
                                    date = lst.find('time', attrs = {'aria-hidden':'true'}).text # 변수명 할당
                                    print(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                    date_list.append(date)
                                except:
                                    print('리뷰 날짜 없음')
                                    date_list.append('')
                        except:
                            pass
                        
                    except:
                        print('리뷰없음')
                        nic_list.append('')
                        re_list.append('')
                        date_list.append('')
                        img_link = ''

                except:
                    print('리뷰 클릭 안됨')
                    nic_list.append('')
                    re_list.append('')
                    date_list.append('')
                    img_link = ''

            else:
                print('가게없음')
                name = naver_df.iloc[i]['콘텐츠명']
                cate = ''
                star = ''
                addr = ''
                open_close = ''
                tel = ''
                
        except:
            try:
                naver_driver.switch_to.default_content()
                naver_driver.switch_to.frame('searchIframe')
                naver_driver.implicitly_wait(3)

                # 가게이름 여러개 중 일치하는 것만 클릭
                try:
                    naver_driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/div[2]/a[1]').click()
                    
                    naver_driver.switch_to.default_content()
                    naver_driver.switch_to.frame('entryIframe')
                    naver_driver.implicitly_wait(3)
                    
                    # 가게이름
                    if naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text == naver_df.iloc[i]['콘텐츠명']:

                        naver_driver.implicitly_wait(3)
                        name = naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text
                        print(name)

                        # 카테고리 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            cate = naver_driver.find_element(By.CLASS_NAME,'DJJvD').text
                            naver_driver.implicitly_wait(3)
                            print(cate)
                        except:
                            print('카테고리없음')
                            cate = ''

                        # 별점 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
                            naver_driver.implicitly_wait(3)
                            print(star)

                        except:
                            print('별점없음')
                            star = ''

                        # 주소 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            addr = naver_driver.find_element(By.CLASS_NAME,'LDgIH').text
                            naver_driver.implicitly_wait(3)
                            print(addr)

                        except:
                            print('주소없음')
                            addr = ''

                        # 운영시간 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/a').send_keys('\n')
                            naver_driver.implicitly_wait(3)
                            oper = naver_driver.find_element(By.CLASS_NAME,'i8cJw').text
                            print(oper)
                            oper_time = naver_driver.find_element(By.CLASS_NAME,'H3ua4').text
                            print(oper_time)
                            open_close = oper+''+oper_time

                        except:
                            print('운영시간없음')
                            open_close = ''

                        # 전화번호 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            tel = naver_driver.find_element(By.CLASS_NAME,'xlx7Q').text
                            naver_driver.implicitly_wait(3)
                            print(tel)

                        except:
                            print('전화번호없음')
                            tel = ''

                        # 리뷰 수집
                        try:
                            naver_driver.implicitly_wait(3)
                            naver_driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]").click()
                            print('리뷰 클릭')

                            # 더보기 클릭
                            try:
                                while True:
                                    naver_driver.implicitly_wait(3)
                                    naver_driver.find_element(By.CLASS_NAME,'fvwqf').click()
                                    print('더보기 클릭')

                                    try:
                                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                        while True:
                                            naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                                            time.sleep(2)

                                            scroll_height = naver_driver.execute_script("return document.body.scrollHeight")

                                            if scroll_location == scroll_height:
                                                break

                                            else:
                                                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                                            naver_driver.implicitly_wait(3)
                                            print('스크롤 내림')

                                    except:
                                        print('스크롤 안됨')

                                    if not naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]'):
                                        break

                            except:
                                print('더보기 안됨')

                            # 리뷰 더보기 클릭
                            try:
                                for button in naver_driver.find_elements(By.CSS_SELECTOR,'div.ZZ4OK > a.xHaT3'):
                                    button.send_keys('\n')
                                    naver_driver.implicitly_wait(3)
                                    print('리뷰 더보기 클릭')

                            except:
                                print('리뷰 더보기 없음')

                            # 이미지, 리뷰 소스 가져오기
                            try:
                                naver_driver.find_element(By.CLASS_NAME,'sBWyy')
                                naver_driver.implicitly_wait(3)
                                print('sBWyy 있음')

                                naver_html = naver_driver.page_source
                                naver_soup = BeautifulSoup(naver_html, 'lxml')
                                naver_driver.implicitly_wait(3)
                                print('페이지 소스 가져옴')

                                # 이미지 수집
                                try:
                                    img = naver_soup.find('div', attrs = {'id':'ibu_1'})
                                    naver_driver.implicitly_wait(3)
                                    img_link = re.findall('["](https.*[.].*g)["]', str(img))
                                    print(img_link)


                                except:
                                    print('이미지없음')
                                    img_link = ''

                                try:
                                    for lst in naver_soup.find_all('li', attrs = {'class':'YeINN'}):
                                        try:
                                            nic = lst.find('div', attrs = {'class':'sBWyy'}).text
                                            print(lst.find('div', attrs = {'class':'sBWyy'}).text)
                                            nic_list.append(nic)

                                        except:
                                            print('닉네임 없음')
                                            nic_list.append('')

                                        try:
                                            naver_driver.implicitly_wait(3)
                                            re = lst.find('span', attrs = {'class':'zPfVt'}).text
                                            print(lst.find('span', attrs = {'class':'zPfVt'}).text)
                                            re_list.append(re)

                                        except:
                                            print('글 리뷰없음')
                                            re_list.append('')

                                        try:
                                            naver_driver.implicitly_wait(3)   
                                            date = lst.find('time', attrs = {'aria-hidden':'true'}).text
                                            print(lst.find('time', attrs = {'aria-hidden':'true'}).text)
                                            date_list.append(date)

                                        except:
                                            print('리뷰 날짜 없음')
                                            date_list.append('')
                                except:
                                      pass

                            except:
                                print('리뷰없음')
                                nic_list.append('')
                                re_list.append('')
                                date_list.append('')
                                img_link = ''


                        except:
                            print('리뷰 클릭 안됨')
                            nic_list.append('')
                            re_list.append('')
                            date_list.append('')
                            img_link = ''

                            
                    else:
                        print('가게없음')
                        name = naver_df.iloc[i]['콘텐츠명']
                        cate = ''
                        star = ''
                        addr = ''
                        open_close = ''
                        tel = ''

                except:
                    print('가게없음')
                    name = naver_df.iloc[i]['콘텐츠명']
                    cate = ''
                    star = ''
                    addr = ''
                    open_close = ''
                    tel = ''
                    
            except:
                print('가게 정보 없음')
                name = naver_df.iloc[i]['콘텐츠명']
                cate = ''
                star = ''
                addr = ''
                open_close = ''
                tel = ''

    except:
        print('가게 정보 없음')
        name = naver_df.iloc[i]['콘텐츠명']
        cate = ''
        star = ''
        addr = ''
        open_close = ''
        tel = ''
        
    for data in range(len(nic_list)):
        row_list =[]
        row_list.append(name)
        row_list.append(cate)
        row_list.append(star)
        row_list.append(addr)
        row_list.append(tel)
        row_list.append(open_close)
        row_list.append(nic_list[data])
        row_list.append(re_list[data])
        row_list.append(date_list[data])
        row_list.append('네이버')
        row_list.append(img_link)
        all_data.append(row_list)

이번에 찾을 키워드 : 0 / 99 행 제주도%20해녀3대할망네
entryIframe 체크
키워드4해녀3대할망네
Fc1rA 있음
해녀3대할망네
해물,생선요리
별점
4.15/5
제주 서귀포시 대정읍 마라로77번길 3
운영시간없음
064-792-4124
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170630_80%2F1498797576669hK0qz_JPEG%2F186580536956354_0.jpg']
먹고즐기고
마라도 뷰맛집입니다. 사장님 너무 친절하시고 바람이 엄청 부는 날인데 멋진 바다 풍경을 보면서 커피 한 잔하기 좋은 카페네요. 날이 좋으면 전망이 더 젛겠지만 오늘은 추위를 피하는 정도로만 .  
2.3.금
리코타치즈
마라도 갔다가 차한잔 마시고 싶었는데
마침 카페가^^
따뜻하게 잘 마

👍👍
21.4.27.화
활력넘치는하루
리뷰보고 그나마 나은 것 같아서 갔는데
전복 짜장. 짬뽕 별로에요
동네 짜장면 집에 비하면 너무 맛이 없어요
짬뽕에 홍합 안싱싱해서 냄새 났어요
해물모듬은 싱싱해요
20.11.29.일
ham****
글 리뷰없음
21.3.9.화
삶이맛나
<분위기에 속아 돈낭비 하지 말자!>
해물 톳 짜장 고소하고 맛있다.
해물 짬뽕도 국물이 시원하고 면발도 무지 맛있다!
모듬해물 한 접시 싱싱하고 쫄깃하다! +제주막걸리😍
다 맛있었음 😋

역시 횟집에서 운영하는 곳이라 그런지 짜장,짬뽕에 들어간 해산물들이 모두 싱싱하고 푸짐하다! 
전체적으로 다 맛있어서 다음에 마라도 오면 또 방문할 것이다!  맛도 가성비도 비주얼도 사장님의 친절함 모두 대 만 족 ✨ 마라도 맛집으로 추천해요~

#마라도여행 #마라도맛집 #마라도짜장면
#마라도톳짜장 #톳짜장 #해물짬뽕 #마라도횟집
#서바당횟집 #마라도서바당횟집
20.6.15.월
수호맘7066
굿
21.5.23.일
모닝키스
굿
21.4.30.금
따이훈
회가 싱싱합니다
20.10.29.목
kal****
#마라도짜장면
마라도짜장면 ₩9,000
#서바당횟집 
기본 짜장면
20.7.17.금
구리8358
장소가 장소인지라 그런지 맛있어요!
직월들 모두 참 친절 하시고요!
20.8.13.목
boy****
해산물을 가게 입구에서 주문즉시 손질해주시는데 신선함이 남달랐어요~ 짬뽕 짜장면도 많이 드시던데 다음 방문때는 짬뽕 먹어보러구요!
20.12.21.월
abirules
솔직 리뷰입니당. 12시 반쯤 들어가서 전복 짜장면과 전복짬뽕을 먹었습니다. 블로그랑 네이버평이 좋아서 일부러 서바당 횟집을 찾아갔어요. 사장님이 엄청 바빠보이셨고, 다른 테이블이 추가로 짜장면을 시킬때 언짢아하셨고, 또 다른 테이블이 공기밥을 주문했을 때 다 떨어졌다고 하셨고 (손님 몰리는 점심시간인데도 불구하고), 저희 테이블이 전복 물회를 추가로 주문하려고 하자 안된다고 하셨습니다. 톳짜장면 양은 그리 많지는 않았지만 맛있었고, 전복 짬뽕도 다른 곳

entryIframe 체크
키워드4강정해녀의집
Fc1rA 있음
강정해녀의집
해물,생선요리
별점
4.28/5
제주 서귀포시 말질로 270 1층
운영시간없음
064-739-0772
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
이미지없음
닉네임 없음
이전 방문기 기억이 너무 좋아 재방문했는데요. 결론만 말씀드리면 재방문은 이제 없습니다. 무엇보다 맛있었던 갱이죽 이제 안하신대요. 이전엔 친구들과 방문했고 이번엔 가족여행중 방문했는데 너무 쌀쌀맞고 서비스가 별로여서 기분이 상했습니다. 사장님. 기분이 나쁜 일이 있으셨는진 모르겠지만 아무리 맛있는 음식을 내와도 그런 서비스를 받고 갈 사람들이 얼마나 있을지 모르겠네요. 
1.21.토
닉네임 없음
네이버 검색 결과보다 가격이 올랐어요. 맛있어요 
22.11.26.토
닉네임 없음
감사합니다 감사합니다 
1.4.수
닉네임 없음
다좋아요
다 맛있고 ^^~
22.11.4.금
닉네임 없음
너무 친절하고 음식도 맛있어요
서귀포 바다가 보이는 음식점
22.7.21.

쌀국수 맛있어요
22.3.23.수
바느
날씨 넘 안좋아서 포장했는데도 맛나요ㅠㅠ
22.1.13.목
Jjj9024
우중충한 날 쌀국수가 땡겨서 혼자 먹으러 갔어요. 점심시간대 피해서 가니까 매장에 손님도 한팀정도 있었고.. 조용히 쌀국수에 집중해서 먹을 수 있었습니다.
숙주 듬뿍,  파도 듬뿍, 고기도 두툼하니 여러 조각 있었어요. 레몬즙을 더 요청했는데  조그마한 그릇에 레몬 슬라이스 한 조각을 꽂아서 주셨어요. 직원분 센스 최고세요👍 덕분에 맛있게 먹었습니다! 

아쉬운점은 계속 먹다보면 좀 짜기도 하고... 마늘후레이크가 기름져서 막판에 조금 느끼한 감 있어요. 베트남 쌀국수의 깔끔한 맛보단 국물 진하고 기름진 맛? ㅎㅎ
21.11.6.토
SUH2
.
22.4.26.화
리샨520929
맛있어요  좌석이 불편하고 덥네요
22.7.29.금
ca****
내부 인테리어가 오묘하지만 매력있고 맛도좋았어요
22.3.23.수
on_****
맛있어요
22.6.12.일
jj7963
맛있어요8
22.5.9.월
선한홍
스프링롤 맛있어요. 볶음밥은 무난했어요. 
바형 테이블이 있어서 혼밥손님도 오기 좋아요. 
점심피크때는 손님이 몰려서 미리 문의하고 오는게 좋아요.
21.10.19.화
사랑이466
맛난곳
22.6.6.월
rud****
홀매니저분이 바쁘신지 주문이 좀 늦었습니다 ㅠ 그것말고 나머지는 다 훌륭했습니다!
22.3.3.목
엠투91
분위기 좋아요
22.5.29.일
aubm
쌀국수만 만족  ㆍ다른메뉴는 별루에요ㆍ
22.3.13.일
blackbean
음식간이 좀 짠편인데 이건 사람입맛따라 다르다쳐도 메뉴 3개 시켰는데 처음나온음식과 간격이 거의30분 걸려 나왔네요. 음식 안나오냐고 문의도 드렸는데 양해 말씀 하나 없이 늦게 주문한 옆테이블 음식이 먼저 나온거도 기분 상했구요.
22.1.23.일
isawara
육지서 아들 오면 아들이 젤 먼저 찾는 곳이라 꼭 가요~~ 언제가도 맛있어요~♡ 이번엔 타무라가 너무 유명해졌는지 30분이나 대기했어요. 나만 알고싶은 맛집인디~♡♡♡


분위기좋아요
20.11.18.수
cocomint21
갈릭쉬림프 맛있었어요 ^^
20.12.26.토
dueleldi
맛있어요~~
특별한 식사^^
21.3.4.목
잉지50
진짜
맛있어요
길 바로 옆이라 지나치기
쉬운데 잘 찾아가세요
20.11.7.토
아인이호
음식이 늦게나오긴하지만 ㅎㅎ 기다릴 맛이였어요 ~ 맛있어용 쌀국수보당 팟타이에 분짜 추천이영
20.7.22.수
이이1259
너무맛있어요!!!!
20.12.26.토
빈말못함
글 리뷰없음
20.11.26.목
11님님님
맛있어요  태국맛집
20.12.15.화
Ar25
팟타이 생각나면 가끔가는데 오늘도 맛있게 먹었습니다~ 근데 비싼 푸팟퐁커리 가격에 비해 '게'가 너무 부실해요... 마른 게다리 몇개랑 살이 없는 게몸통 두개가 전부였습니다.
20.10.15.목
리리5645
팟타이가 맛이 있다 해서  먹어봤는데 코코넛오일의 향이 거의 없었다... 왜 맛있다고 하는건지 궁금한....
분짜는.. 쌀국수 소면이였으면 좋았을꺼 같은데... 소스가 별로다... 면도 바로 삶아서 나온거 같아 보이지 않았다.  왜 삼겹살을 사용한지 모르겠고 불맛을 내기 위한 그을림이 약간 거슬렸다. 분짜는 고기가 주가 아닌데.... 차라리 고기 양을 줄이고 가격을 내렸으면.... 맛이 있다 해서 기대 하고 갔는데 실망하고 왔다...
베트남던 태국이던 인니던 하나만 전문적으로 했으면 하는 아쉬움이 있다.. 테이블 역시 불편.....
20.7.5.일
랄랄라4740
글 리뷰없음
20.12.20.일
토토814
갈때마다 맛있게 먹고 옵니다
그런데 홀에
직원분이
부족해서 항상 대기가 너무 길어요 ㅠ
20.8.21.금
JERRYKIM
친절하고 좋습니다.
20.11.7.토
별명뭐하지86
으엄청 맛있고 재방문의사 1000000프로
20.10.10.토
비로올게
잘 먹었어요
20.12.5.토
네깜맘
팟타이 맛있어요
20.8.21.금
빛나는빛
맛나요
20.12.15.화
나요니요
사태랑 새우는 맛있는데 팟타이 향신료 향이 너무 강해서 다 못먹었네요
20.8.9.일
저스

...

21.1.18.월
권민희38
택시 타고 다녀왔어요 너무 예쁘고 맛있었어요!!!!!
20.10.23.금
jejuj
분위기는 좋으나, 맛은 좀 별로...
20.10.24.토
Ahwiiqn
어디 숨어 있더라도 찾아가고 싶은 곳이네요😄
20.10.21.수
sakura ko
디쉬 하나하나 맛있었습니다. 
특히 바다이야기 ^^
조용하고 차분한 저녁식사였기에 가족모두 만족한 시간이었습니다. 
20.8.28.금
807tu
디테일이 살아 있는 인테리어부터 맛있는 와인과 요리까지, 전부 만족스러웠어요. 특히 뇨끼는 정말 must try예요.
20.7.5.일
jenny0801
제주문어보말파스타 맛있어요.
성게어란파스타도 맛나네요.^^

20.8.15.토
ziz****
맛과 분위기 모두 좋았어요. 운전때문에 남편은 와인은 못마셨지만 즐거운 시간 보냈습니다 :) 테이블에 물병을 따로 주시거나 물이 떨어지면 바로 리필을 해주시면 더 좋을 것 같아요!
20.9.13.일
파티맘 별
분위기도 좋고. 맛은 더 좋아요
20.9.13.일
ra****
너무너무 맛있었어요.
이태리 여행온듯한 느낌.
말그대로 트라또리아.
분위기 완전 좋았구요.
딱새우로제파스타 추천합니다
20.8.1.토
갸하하1
피자가 맛있어요! 다음에 간다면 마르게리따를 먹고싶네요 ㅠㅠ
20.9.9.수
허정남
분위기가 세상에 마치 유럽에온듯했어요
20.10.3.토
이번에 찾을 키워드 : 44 / 99 행 제주도%20푸더매%20제주
entryIframe 체크
가게없음
이번에 찾을 키워드 : 45 / 99 행 제주도%20카페2085
entryIframe 체크
카페2085
브런치
별점없음
제주 제주시 조천읍 선교로 185 (선흘리 2085번지)
운영시간없음
0507-1381-1729
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 

사장님 너무 장인정신이 돋보이시고,
비빔막국수 너어어어어어무 마싯어서
꼭 다시 갈꺼에용,,,❤️
20.9.8.화
콩닭콩닭
비빔막국수 육전온국수 돔베고기 
정말 맛있게 먹었어요  사장님 너무 친절하시고
음식에서 정성이 느껴졌어요
사업 번창하세요
20.8.11.화
mns****
메밀국수
20.11.13.금
미니462
돔베고기가 대박!!!!
20.9.5.토
ara****
저 원래 비빔 안 조아하고 물 조아하거둔용 ,, 근데 비빔이 참 마싯네영,,,❤️
20.9.8.화
gtn****
기다리는 시간이  있지만 맛은 강추입니다
20.8.23.일
낭만네꼬
건강한 맛~~
20.9.13.일
몽상가6231
역시 여름은 비빔이죠!!!
그런데 시원한 물막도 거부 못하겠고...
고소한 육전이 올라오는 온막국수는 진짜...
암튼 오늘은 고민 많았지만...
비빔으로...

잘 먹었습니다...^^
20.7.16.목
루시아5778
글 리뷰없음
20.8.15.토
thy****
월요일은 안하니까 주의하세요
20.7.2.목
jej****
비빔막국수 최고입니다.
20.7.16.목
이번에 찾을 키워드 : 54 / 99 행 제주도%20테이블앤데스크
entryIframe 체크
가게없음
이번에 찾을 키워드 : 55 / 99 행 제주도%20수망일기
entryIframe 체크
가게없음
이번에 찾을 키워드 : 56 / 99 행 제주도%20종달해녀의집
entryIframe 체크
가게없음
이번에 찾을 키워드 : 57 / 99 행 제주도%20면사무소
entryIframe 체크
면사무소
칼국수,만두
별점
4.5/5
제주 제주시 애월읍 애월해안로 494
운영시간없음
0507-1332-1402
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 내림
더보기 클릭
스크롤 

20.7.18.토
다댐벼아뵤
#애월칼국수
20.9.7.월
엠케이님
뷰도 좋고, 맛있어요! 배부르게 잘 먹었어요~
20.6.29.월
소리샘6960
2인셋트 시켰어요
20.8.24.월
sou****
넘넘  맛있어요ㆍ
맛집으로 등록
20.8.24.월
쭈니들맘22
좋아요
20.11.5.목
swe****
맛있어요
20.11.5.목
chan000926
맛있어요
20.10.14.수
에티오피아
만족도 높은 음식과 바다 전망
20.3.31.화
cj****
좋아용
20.8.6.목
tigress22
맛있어요
20.9.12.토
블루점프
맛있어요
20.4.25.토
이번에 찾을 키워드 : 58 / 99 행 제주도%20홍삼흑돼지
entryIframe 체크
가게없음
이번에 찾을 키워드 : 59 / 99 행 제주도%20까떼커피
entryIframe 체크
가게없음
이번에 찾을 키워드 : 60 / 99 행 제주도%20체면
entryIframe 체크
가게없음
이번에 찾을 키워드 : 61 / 99 행 제주도%20보라네
entryIframe 체크
보라네
한식
별점없음
제주 제주시 추자면 추자로 96
운영시간없음
064-744-4305
리뷰 클릭
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
이미지없음
아름다운날들60
사장님 친절하시고 반찬들이 너무 맛있어요!
제육볶음도 짱맛!!
22.6.2.목
하늘사랑89
음식이 신선하고 맛있어요~
22.5.25.수
rla****
정식도 넘 좋아요
22.4.29.금
보라돌이5048
반찬들이 다 맛있었어요. 만원에 생선구이랑 계란후라이랑 집반찬 나와요
21.11.20.토
네맘대로살아라
좋아여
21.10.23.토
jie73kr
굿
21.7.15.목
이번에 찾을 키워드 : 62 / 99 행 제주도%20우도향
entryIframe 체크
가게없음
이번에 찾을 키워드 : 63 / 99 행 제주도%20만부정
entryIframe 체크
가게없음
이번에 찾을 키워드 : 64

entryIframe 체크
가게없음
이번에 찾을 키워드 : 75 / 99 행 제주도%20카페%20소바니
entryIframe 체크
가게없음
이번에 찾을 키워드 : 76 / 99 행 제주도%20카페더몹시
entryIframe 체크
가게없음
이번에 찾을 키워드 : 77 / 99 행 제주도%20유기농커피%20카페다
entryIframe 체크
가게없음
이번에 찾을 키워드 : 78 / 99 행 제주도%20해와달그리고섬
entryIframe 체크
가게없음
이번에 찾을 키워드 : 79 / 99 행 제주도%20섶섬지기
entryIframe 체크
가게없음
이번에 찾을 키워드 : 80 / 99 행 제주도%20해성도뚜리
entryIframe 체크
가게없음
이번에 찾을 키워드 : 81 / 99 행 제주도%20루스트플레이스%20칠성로점
entryIframe 체크
가게없음
이번에 찾을 키워드 : 82 / 99 행 제주도%20옛정의골%20식당
entryIframe 체크
가게없음
이번에 찾을 키워드 : 83 / 99 행 제주도%20블루하우스
entryIframe 체크
가게없음
이번에 찾을 키워드 : 84 / 99 행 제주도%20홍도니
entryIframe 체크
홍도니
돼지고기구이
별점
3.7/5
제주 제주시 연북로 187
운영시간없음
064-752-1294
리뷰 클릭
더보기 클릭
스크롤 내림
더보기 안됨
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
리뷰 더보기 클릭
sBWyy 있음
페이지 소스 가져옴
이미지없음
사적인견해
해장국에는 선지와 소고기가 많습니다.
한국 성인 아침 식사로 든든하고 좋습니다.
개인적으로 대파김치가 깍두기보다 맛있습니다.
22.10.2.일
불량꿀꿀이
여기 밑반찬 다 맛있어요
음식 좀 하는 집이라는 뜻ㅋ
메인 음식은 양념을 좀 걷어내고 먹었어야는데
그대로 먹었더니


entryIframe 체크
가게없음


In [27]:
all_data

[['해녀3대할망네',
  '해물,생선요리',
  '별점\n4.15/5',
  '제주 서귀포시 대정읍 마라로77번길 3',
  '064-792-4124',
  '',
  '먹고즐기고',
  '마라도 뷰맛집입니다. 사장님 너무 친절하시고 바람이 엄청 부는 날인데 멋진 바다 풍경을 보면서 커피 한 잔하기 좋은 카페네요. 날이 좋으면 전망이 더 젛겠지만 오늘은 추위를 피하는 정도로만 .  ',
  '2.3.금',
  '네이버',
  ['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170630_80%2F1498797576669hK0qz_JPEG%2F186580536956354_0.jpg']],
 ['해녀3대할망네',
  '해물,생선요리',
  '별점\n4.15/5',
  '제주 서귀포시 대정읍 마라로77번길 3',
  '064-792-4124',
  '',
  '리코타치즈',
  '마라도 갔다가 차한잔 마시고 싶었는데\n마침 카페가^^\n따뜻하게 잘 마시고 나왔습니다',
  '1.22.일',
  '네이버',
  ['https://search.pstatic.net/common/?autoRotate=true&amp;type=w560_sharpen&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20170630_80%2F1498797576669hK0qz_JPEG%2F186580536956354_0.jpg']],
 ['해녀3대할망네',
  '해물,생선요리',
  '별점\n4.15/5',
  '제주 서귀포시 대정읍 마라로77번길 3',
  '064-792-4124',
  '',
  '골드만 싹슬이',
  '커피 맷 최고',
  '2.6.월',
  '네이버',
  ['https://search.pstatic.net/common/?autoRotate=true&amp;type=w

In [28]:
column_name = ['STORE_NAME', 'CATEGORY','STORE_SCORE', 'STORE_ADDR', 'TEL', 'OPEN_CLOSE_TIME', 'WRITER','CONTENT', 'CREATED_AT', 'TYPE', 'IMG_LINK']
df2 = pd.DataFrame(
    all_data, columns = [column_name])
df2

,STORE_NAME,CATEGORY,STORE_SCORE,STORE_ADDR,TEL,OPEN_CLOSE_TIME,WRITER,CONTENT,CREATED_AT,TYPE,IMG_LINK
0,해녀3대할망네,"해물,생선요리",별점\n4.15/5,제주 서귀포시 대정읍 마라로77번길 3,064-792-4124,,먹고즐기고,마라도 뷰맛집입니다. 사장님 너무 친절하시고 바람이 엄청 부는 날인데 멋진 바다 풍...,2.3.금,네이버,[https://search.pstatic.net/common/?autoRotate...
1,해녀3대할망네,"해물,생선요리",별점\n4.15/5,제주 서귀포시 대정읍 마라로77번길 3,064-792-4124,,리코타치즈,마라도 갔다가 차한잔 마시고 싶었는데\n마침 카페가^^\n따뜻하게 잘 마시고 나왔습니다,1.22.일,네이버,[https://search.pstatic.net/common/?autoRotate...
2,해녀3대할망네,"해물,생선요리",별점\n4.15/5,제주 서귀포시 대정읍 마라로77번길 3,064-792-4124,,골드만 싹슬이,커피 맷 최고,2.6.월,네이버,[https://search.pstatic.net/common/?autoRotate...
3,해녀3대할망네,"해물,생선요리",별점\n4.15/5,제주 서귀포시 대정읍 마라로77번길 3,064-792-4124,,대박7366,분위기가 좋아요,22.9.23.금,네이버,[https://search.pstatic.net/common/?autoRotate...
4,해녀3대할망네,"해물,생선요리",별점\n4.15/5,제주 서귀포시 대정읍 마라로77번길 3,064-792-4124,,xhd****,굿,22.8.22.월,네이버,[https://search.pstatic.net/common/?autoRotate...
...,...,...,...,...,...,...,...,...,...,...,...
3853,메로식당,"해물,생선요리",별점\n4.61/5,제주 제주시 광양13길 34,064-757-1411,월07:30 - 21:00,황금박쥐1942,조미료 많이 들어가지 않은 음식~ 맛있어요,20.8.6.목,네이버,
3854,메로식당,"해물,생선요리",별점\n4.61/5,제주 제주시 광양13길 34,064-757-1411,월07:30 - 21:00,valiant11,JMT,20.11.10.화,네이버,
3855,메로식당,"해물,생선요리",별점\n4.61/5,제주 제주시 광양13길 34,064-757-1411,월07:30 - 21:00,휴가원8242,#메로지리 #메로구이 #제주메로 #메로 #현지인식당\n\n메로구이 추천,20.8.4.화,네이버,
3856,메로식당,"해물,생선요리",별점\n4.61/5,제주 제주시 광양13길 34,064-757-1411,월07:30 - 21:00,e_m****,도다리쑥국이 먹고 힘내고싶었던 날인데 메로지리로 먹고 땀내고 힘냈어요 지리에 배추와...,20.3.19.목,네이버,


In [29]:
naver_driver.close()
df2.shape

(3858, 11)

In [30]:
df2.to_csv('naver_restorant_review3.csv',index=False, encoding='utf-8')